In [6]:
# import pandas

import pandas as pd

In [7]:
# Load Data

site_list = pd.read_excel(r"C:\Users\lesli\github-classroom\Shell_Excentus_Files\Shell_Excentus\Shell_Site_List.xlsx")
monthly_excentus_download = pd.read_excel(r"C:\Users\lesli\github-classroom\Shell_Excentus_Files\Shell_Excentus\Site_Redeemer_Issuer_Settlement_3160398.xlsx", skiprows=1)
previous_vendor_discount = pd.read_excel(r"C:\Users\lesli\github-classroom\Shell_Excentus_Files\Shell_Excentus\March_Vendor_Discount.xlsm")

In [8]:
# Raname Columns

site_list.rename(columns={"Merchant_Id_1":"Site ID"}, inplace=True)

In [10]:
# Drop specified columns

site_list.drop(columns={"Zip Code"}, inplace=True)

In [11]:
# Convert Site ID to string type

site_list["Site ID"] = site_list["Site ID"].astype(str)

In [12]:
# Drop last two rows with NAN that only show totals

monthly_excentus_download.drop(monthly_excentus_download.tail(2).index, inplace=True)

In [13]:
# Drop specified columns
columns_to_drop = ["Post Date", "Site Name", "Total Discounted Gallons", "Total Discounts Redeemed", 
"Site Funded", "Redemption Fee", "Issuance Fee", "Flat Fee", "Program ID", "Type"]
monthly_excentus_download.drop(columns=columns_to_drop, inplace=True)

In [14]:
# Convert Site ID to string type

monthly_excentus_download['Site ID'] = monthly_excentus_download['Site ID'].astype(str)

In [15]:
# Remove .0 from Site ID

monthly_excentus_download['Site ID'] = monthly_excentus_download['Site ID'].astype(str).apply(lambda x: x[:-2] if x.endswith('.0') else x)

In [16]:
# Process previous_vendor_discount :  Rename Column Names Previous Vendor Funded Discount

previous_vendor_discount.rename(columns={"Site ID": "Customer Name", "Site Name": "Customer #"}, inplace=True)

In [17]:
# Group monthly excentus by site ID and sum

grouped_monthly_excentus = monthly_excentus_download.groupby("Site ID").sum().reset_index()

In [18]:
# Merge Site List to Grouped Monthly Excentus

merged_site_list_excentus = pd.merge(site_list, grouped_monthly_excentus, on= "Site ID", how="outer")

In [19]:
# Sort Grouped Excentus by Customer #
sorted_merged_excentus = merged_site_list_excentus.sort_values(by="Customer #")

In [20]:
# Create New DF for Current Vendor Discount
current_vendor_discount = sorted_merged_excentus[["Site ID", "Customer #", "Vendor Funded Discounts"]]

In [21]:
# Export vendor_discount_new to Excel

current_vendor_discount.to_excel("May_Vendor_Funded_Discounts.xlsx", index=False)

In [22]:
# Drop Current Month Vendor Funded Discounts

current_month_payables_excentus = sorted_merged_excentus.drop(columns=["Vendor Funded Discounts"], inplace=False)

In [23]:
# Rename columns in Previous Vendor Discount

previous_vendor_discount.rename(columns={"Site Name": "Customer #"}, inplace=True)

In [24]:
# Drop NAN Rows

previous_vendor_discount = previous_vendor_discount.dropna()

In [25]:
# Merge current_month_payables_excentus with previous_vendor_discount

merged_excentus_vd = pd.merge(previous_vendor_discount,current_month_payables_excentus,on="Customer #",how="outer",suffixes=('_new', '_curr'))

In [26]:
# Fill in NAN with 0 in Vendor Funded Discounts Column

merged_excentus_vd["Vendor Funded Discounts"] = merged_excentus_vd["Vendor Funded Discounts"].fillna(0)

In [27]:
# Round numbers to two decimal points
columns_to_round = ["Vendor Funded Discounts", "Total Receivable - Daily", "Total Payable - Daily"]
merged_excentus_vd[columns_to_round] = merged_excentus_vd[columns_to_round].round(2)

In [28]:
# Drop Duplicate Columns

merged_excentus_vd_dropped = merged_excentus_vd.drop("Customer Name_new", axis=1, inplace=False)

In [29]:
# Rename Columns
merged_excentus_vd_renamed = merged_excentus_vd_dropped.rename(columns={"Customer Name_curr": "Customer Name",  "Total Payable - Daily": "Total Payable"}, inplace=False)

In [30]:
# Add Vendor Funded Discount to Total Receivable
merged_excentus_vd_renamed["Total Receivable"] = merged_excentus_vd_renamed["Total Receivable - Daily"] + merged_excentus_vd_renamed["Vendor Funded Discounts"]

In [31]:
# Drop Columns and Create Final Total Sums Sheet
columns_to_drop = ['Vendor Funded Discounts', 'Total Receivable - Daily']
total_sum_excentus = merged_excentus_vd_renamed.drop(columns=columns_to_drop, inplace=False)


In [32]:
# Reorder Columns for Entry
new_order = ["Site ID", "Customer Name", "Customer #", "Total Receivable", "Total Payable"]
pre_mas_entry_excentus = total_sum_excentus[new_order]
pre_mas_entry_excentus

,Site ID,Customer Name,Customer #,Total Receivable,Total Payable
0,57444687107,a3,60-A-1001,761.86,393.39
1,57445661903,a4,60-A-1002,337.24,150.41
2,12823811000,a5,60-A-1003,432.36,212.12
3,12625514000,a8,60-A-1004,2576.37,1266.22
4,12746116008,a9,60-A-1005,278.73,139.48
5,57444432801,a11,60-A-1006,2044.32,1151.34
6,12813645004,a12,60-A-1007,350.07,118.66
7,57444139208,a13,60-A-1008,3986.29,2244.83
8,12580077001,a14,60-A-1009,813.52,421.68
9,57444434302,a15,60-A-1010,972.47,523.00


In [33]:
# Calculate totals across numerical columns
totals = pre_mas_entry_excentus.select_dtypes(include=[float, int]).sum()
totals

Total Receivable    37354.76
Total Payable       19615.81
dtype: float64

In [34]:
totals_df = pd.DataFrame(totals).T
totals_df.index = ['Totals']  # Set index label for the totals row

# Concatenate totals_df with pre_mas_entry_excentus
df_totals = pd.concat([pre_mas_entry_excentus, totals_df])

In [35]:
df_totals.to_excel("May_2024_Excentus.xlsx", index=False)
